In [ ]:
import sys
sys.path.append('./clamp/lux/pyuid3')
sys.path.append('./clamp/lux/lux')

In [ ]:
from clamp.clamp import *
import sklearn
import pandas as pd
import numpy as np
#import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import pickle
import re
import math
import warnings
import os
import math
import matplotlib.pyplot as plt
import socket
import tempfile
warnings.filterwarnings('ignore')

In [ ]:
'''
In console run:
java -jar HMRServer.jar <numer_portu> <ilosc_watkow> e.g.
java -jar HMRServer.jar 9999 24
'''

HOST = "127.0.0.1" #"127.0.0.1"  # The server's hostname or IP address
PORT = 9999 # The port used by the server

def queryHRTDServer(query,max_msg_size=1024):
    query+='\n'
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(bytes(query,'UTF8'))
        fragments = []
        while True: 
            chunk = s.recv(max_msg_size)
            if not chunk: 
                break
            fragments.append(chunk)
        arr = b''.join(fragments)
    return arr.decode('UTF8')

def rem_hmr_files():
    for x in os.listdir():
        if x.endswith(".hmr"):
            os.remove(x)

# Example based on iris dataset without crossvalidation

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
features = ['sepal_length','sepal_width','petal_length','petal_width']
target = 'calss'

#create daatframe with columns names as strings (LUX accepts only DataFrames withj string columns names)
dataset = pd.DataFrame(iris.data,columns=features)
dataset[target] = iris.target


In [ ]:

data = dataset.drop(target, axis = 1)

# labels were removed because one of the clamp's feature is to make clustering, 
# however if necessary you can pass labels in fit function and then clustring stage will be omitted

In [ ]:
data.head()

In [ ]:
clamp = CLAMP(clusterng_algorithm = KMeans(n_clusters = 3), 
                classification_model = xgb.XGBClassifier(), 
                description_points_ratio=5,
                test_size = 0.1,
                metric = 'minkowski',
                thresh = 0.9, 
                bounding_box_selection ='random',
                explainer_type = 'lux',
                approach = 'other')
#parameters to adjust there is also possibility to change clustering algorithm and classification model
# available bounding_box_selection parameter: centroids, outliers, tree_query, random

In [ ]:
#clamp.fit(data, labels) -- this one is for data with labels
clamp.fit(data) # -- this one is for data without labels

#the dataset has been splited in to train and test dataset, train dataset in provided to anchor to generate rules based on boundingbox method and test dataset is saved to run predict method (check below)

In [ ]:
predict = clamp.predict(clamp.X_test, clamp.y_test) #labels generation based on the test dataset in generated rules in previous step

In [ ]:
print(clamp.explainer_accuracy) 
#score calculation (comparison labels from test dataset and those predicted by the clamp -- previous step)
#available: explainer_f1, explainer_accuracy, explainer_classification_report, explainer_recall

In [ ]:
r = clamp.justify()
r
rem_hmr_files() # remove temp hmr files


In [ ]:
#display genarated rules
r

# Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import Birch

In [ ]:
def scorer(clamp, *args):
    clamp.predict(clamp.X_test, clamp.y_test)
    return {'f1': clamp.explainer_f1, 'accuracy': clamp.explainer_accuracy, 'precision': clamp.explainer_precision}

In [ ]:
parameters = [{
                'bounding_box_selection': ['random', 'centroids', 'outliers', 'tree_query'],
                'description_points_ratio': [2, 4, 8, 13],
                'test_size': [0.2],
                'thresh':[0.9],
                'explainer_type': ['lux'],
                'neighborhood_size':  [20],
                'max_depth': [2, 4, 5]
},
            {
                'explainer_type': ['global'],
                'description_points_ratio': [2,4,8,13],
                'test_size': [0.25]
}, 
                { 
                'description_points_ratio': [2, 4, 8, 13],
                'test_size': [0.2],
                'thresh':[0.9],
                'explainer_type': ['lux'],
                'approach': ['other'],
                'neighborhood_size':  [20],
                'max_depth': [2, 4, 5]
}]


In [ ]:
list_of_choosen_datasets = [
    'ecoli.csv',
    'lymphography.csv',
    'glass.csv',
    'balance.csv',
    'breast_tissue.csv',
    'primary_tumor.csv',
    'vote.csv',
    'ionosphere.csv',
    'cmc.csv',
    'bupa.csv']

In [ ]:
list_of_clusters = []
for data in list_of_choosen_datasets:
    list_of_clusters.append(len(pd.read_csv('synthetic_datasets/'+data)['y'].unique()))

In [ ]:
#parameter settings as described above
from sklearn.exceptions import UndefinedMetricWarning
os.environ["PYTHONWARNINGS"] = "ignore::FutureWarning,ignore::UndefinedMetricWarning"
cv_restuls = []
cv_datasets = []

for dataset,cluster_number in zip(list_of_choosen_datasets, list_of_clusters):
    data = pd.read_csv('synthetic_datasets/'+dataset)
    data.dropna(inplace = True)
    data.reset_index(drop = True, inplace = True)
    labels = data['y']
    data = data.drop('y', axis = 1)
    print(f'Dataset: {dataset}')
    
    clamp = CLAMP(clusterng_algorithm = KMeans(n_clusters = cluster_number), classification_model = RandomForestClassifier())
    
    clf = GridSearchCV(clamp, parameters, scoring = scorer, cv = 10, refit = 'accuracy', n_jobs = 20, error_score = 'raise')
    clf.fit(data, labels)
    
    rem_hmr_files()

In [ ]:
rem_hmr_files()